# Model Stacking

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from pathlib import Path
from tensorflow.keras.utils import image_dataset_from_directory
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

test_data_path = Path(r"D:\Virtual Environments\Pattern Recognition\segmented dataset\meta_model_train")
efficientnet_model_path = Path(r"D:\Virtual Environments\Pattern Recognition\Models\EfficientNet")
resnet_model_path = Path(r"D:\Virtual Environments\Pattern Recognition\Models\ResNet")
vgg_model_path = Path(r"D:\Virtual Environments\Pattern Recognition\Models\VGG")

IMAGE_SIZE = (256, 256)
BATCH_SIZE = 8
NUM_CLASSES = 10 

test_ds = image_dataset_from_directory(
    str(test_data_path),
    shuffle=False,
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE
)

#                         Loading base learners
efficientnet_loaded = tf.saved_model.load(str(efficientnet_model_path))
efficientnet_infer = efficientnet_loaded.signatures["serving_default"]

resnet_loaded = tf.saved_model.load(str(resnet_model_path))
resnet_infer = resnet_loaded.signatures["serving_default"]

vgg_loaded = tf.saved_model.load(str(vgg_model_path))
vgg_infer = vgg_loaded.signatures["serving_default"]

#            Generate probability predictions from each base learner
efficientnet_probs = []
resnet_probs = []
vgg_probs = []
true_labels = []

for images, labels in test_ds:
    #                   EfficientNet probabilities
    preds_eff = efficientnet_infer(images)
    if isinstance(preds_eff, dict):
        preds_eff = list(preds_eff.values())[0]
    preds_eff = preds_eff.numpy()  
    
    #                     ResNet probabilities
    preds_resnet = resnet_infer(images)
    if isinstance(preds_resnet, dict):
        preds_resnet = list(preds_resnet.values())[0]
    preds_resnet = preds_resnet.numpy()  
    
    #                     VGG probabilities
    preds_vgg = vgg_infer(images)
    if isinstance(preds_vgg, dict):
        preds_vgg = list(preds_vgg.values())[0]
    preds_vgg = preds_vgg.numpy() 
    
    #           Append probabilities and corresponding true labels
    efficientnet_probs.append(preds_eff)
    resnet_probs.append(preds_resnet)
    vgg_probs.append(preds_vgg)
    true_labels.extend(labels.numpy())

#                    Concatenate all probability arrays
efficientnet_probs = np.concatenate(efficientnet_probs, axis=0)  # Shape: [N, NUM_CLASSES]
resnet_probs = np.concatenate(resnet_probs, axis=0)
vgg_probs = np.concatenate(vgg_probs, axis=0)
true_labels = np.array(true_labels)

# Create a pandas dataframe for stacking using model probabilities as features

efficientnet_cols = [f"EfficientNet_Class_{i}" for i in range(NUM_CLASSES)]
resnet_cols = [f"ResNet_Class_{i}" for i in range(NUM_CLASSES)]
vgg_cols = [f"VGG_Class_{i}" for i in range(NUM_CLASSES)]

df_efficientnet = pd.DataFrame(efficientnet_probs, columns=efficientnet_cols)
df_resnet = pd.DataS

Found 5453 files belonging to 10 classes.
Sample of stacking features with probabilities:
   EfficientNet_Class_0  EfficientNet_Class_1  EfficientNet_Class_2  \
0              0.000059              0.000290          3.303785e-02   
1              0.284511              0.000005          3.872419e-05   
2              0.998579              0.000001          2.129611e-07   
3              0.741073              0.000205          3.983658e-06   
4              0.999323              0.000003          6.519338e-07   

   EfficientNet_Class_3  EfficientNet_Class_4  EfficientNet_Class_5  \
0          1.195257e-02              0.001914          5.050433e-01   
1          4.781166e-05              0.000715          7.009579e-01   
2          1.436287e-04              0.000045          9.428346e-06   
3          4.757738e-07              0.258709          1.089334e-08   
4          1.026797e-06              0.000035          4.204391e-05   

   EfficientNet_Class_6  EfficientNet_Class_7  Efficient

# Meta Model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

NUM_CLASSES = 10  

#           Create feature names for each base model's probability outputs
efficientnet_cols = [f"EfficientNet_Class_{i}" for i in range(NUM_CLASSES)]
resnet_cols = [f"ResNet_Class_{i}" for i in range(NUM_CLASSES)]
vgg_cols = [f"VGG_Class_{i}" for i in range(NUM_CLASSES)]
feature_cols = efficientnet_cols + resnet_cols + vgg_cols

X = stacking_df[feature_cols]
y = stacking_df['TrueLabel']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

meta_model = RandomForestClassifier(random_state=42)
meta_model.fit(X_train, y_train)

# Predict on the evaluation split using the meta model
y_pred = meta_model.predict(X_test)

# Evaluation

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

print("Meta Model Evaluation Metrics:")
print("===============================")
print(f"Accuracy:   {accuracy:.4f}")
print(f"Precision:  {precision:.4f}")
print(f"Recall:     {recall:.4f}")
print(f"F1 Score:   {f1:.4f}")
print("===============================")

Meta Model Evaluation Metrics:
Accuracy:   0.8708
Precision:  0.8729
Recall:     0.8708
F1 Score:   0.8712
